In [1]:
import navis

In [31]:
query = navis.read_nrrd('VFB_001013cg.nrrd', output='dotprops', k=20, threshold=100)
query.id = 'MB566C'
query

,
type,navis.Dotprops
name,VFB_001013cg
id,MB566C
k,20
units,1 micrometer
n_points,10375


In [ ]:
fig = navis.plot3d(query)

In [10]:
sk = navis.read_swc('/Users/philipps/Downloads/hemibrain-v1.2-skeletons/',
                    include_subdirs=True, fmt='{name,id:int}.swc')

Importing:   0%|          | 0/97900 [00:00<?, ?it/s]

In [13]:
sk.sort_values('n_nodes')
sk = sk[:30_000]
sk

,type,name,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units
0,navis.TreeNeuron,425790257,182514,None,20286,20851,1.001564e+07,None,1 dimensionless
1,navis.TreeNeuron,5813105172,159749,None,22416,22866,7.610828e+06,None,1 dimensionless
...,...,...,...,...,...,...,...,...,...
29998,navis.TreeNeuron,5813035385,1018,None,90,100,6.213784e+04,None,1 dimensionless
29999,navis.TreeNeuron,2375870447,1018,None,151,156,3.505048e+04,None,1 dimensionless


In [14]:
# Note that we are resampling to 1 point for every micron of cable 
# Because these skeletons are in 8nm voxels we have to use 1000/8
targets = navis.make_dotprops(sk, k=5, parallel=True, resample=1000/8)

Dotprops:   0%|          | 0/30000 [00:00<?, ?it/s]

In [21]:
import flybrains

query_xf = navis.xform_brain(query, source='JRC2018U', target='JRCFIB2018Fraw')

Transform path: JRC2018U -> JRC2018F -> JRCFIB2018Fum -> JRCFIB2018F -> JRCFIB2018Fraw


In [ ]:
# Note that we convert from voxels (8nm) to microns
scores = navis.nblast(query_xf / 125, targets / 125, scores='mean')

In [33]:
scores.head()

,425790257,5813105172,5813057579,5813050499,859839499,5813039148,1640909284,423101189,1321564092,5813024698,...,2032311172,2283477509,1686751696,2180970798,1904079437,1348510839,5813047115,516672812,5813035385,2375870447
MB566C,-0.793901,-0.870922,-0.767066,-0.627762,-0.612109,-0.591567,-0.827376,-0.878581,-0.680821,-0.832845,...,-0.881125,-0.817378,-0.88186,-0.881415,-0.881596,-0.549413,-0.881259,-0.880987,-0.88139,-0.667554


In [35]:
scores.loc['MB566C'].sort_values(ascending=False)

2030342003    0.218951
2214504597    0.214873
1069223047    0.183628
856131667     0.119994
2214846055    0.094881
                ...   
2056511651   -0.883308
296501634    -0.883329
2275102579   -0.883354
5813091313   -0.883436
910404284    -0.883766
Name: MB566C, Length: 30000, dtype: float64

In [ ]:
fig = navis.plot3d([query_xf, targets.idx[[2030342003, 2214504597, 1069223047]]])

In [40]:
# Remove the left-hand-side neuron based on the position 
# along the x-axis (this is just one of the possible approaches)
# This is the approximate LHS boundary of the volume
flybrains.JRCFIB2018Fraw.mesh.vertices[:, 0].max()

TrackedArray(34499.97265625)

In [41]:
query_ss = navis.subset_neuron(query_xf, query_xf.points[:, 0] <= 35_000)
query_ss

,
type,navis.Dotprops
name,VFB_001013cg
k,20
units,9.999999999999998 nanometer
n_points,5093
